# TWITTER SENTIMENT ANALYSIS

In [122]:
import nltk
import numpy as np
import pandas as pd

Reading the training and testing data

In [123]:
train = pd.read_csv(r'C:\Users\asus\Downloads\TwitterSentiment\train.csv')
test = pd.read_csv(r'C:\Users\asus\Downloads\TwitterSentiment\test.csv')

Selecting the columns required

In [124]:
train_data = train.text
train_target = train.airline_sentiment
test_data = test.text

Data Cleaning with NLTK

In [125]:
from nltk.corpus import stopwords
import string
stop = stopwords.words('english')
stop.remove("isn't")
punctuations = list(string.punctuation)
stop += punctuations

In [126]:
from nltk.tokenize import word_tokenize, TweetTokenizer
#nltk.download('punkt')

In [287]:
tt = TweetTokenizer(strip_handles=False)

In [288]:
documents = []
for i in range(len(train_data)):
    documents.append([train_data[i], train_target[i]])

In [289]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk import wordnet
lemmatizer = WordNetLemmatizer()

In [290]:
def postag(st):
    s = str(st)
    if s.startswith('J'):
        return wordnet.ADJ
    elif s.startswith('N'):
        return wordnet.NOUN
    elif s.startswith('V'):
        return wordnet.VERB
    elif s.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [291]:
def clean_words(doc):
    output_words = []
    words = tt.tokenize(doc)
    for word in words:
        if (word.lower() not in stop) and (word.isdigit() == False):
            pos = pos_tag(word)
            out = lemmatizer.lemmatize(word, pos = postag(pos))
            output_words.append(out)
    return output_words

Forming the document dictionary for training data

In [292]:
documents_new = [(clean_words(doc), target) for doc, target in documents]

In [293]:
testing_data = [clean_words(doc) for doc in test_data]

In [300]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

In [301]:
x_train = [" ".join(doc) for doc, target in documents_new]
y_train = [target for doc, target in documents_new]
x_test = [" ".join(data) for data in testing_data]

In [302]:
# x_tr, x_te, y_tr, y_te = train_test_split(x_train, y_train, random_state = 1)

Importing the Classifiers for sentiment analysis

In [303]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [479]:
Count_Vec = CountVectorizer(max_features= 2890, max_df=0.6, tokenizer= tt.tokenize)#, ngram_range=(1,2))
x_train_v = Count_Vec.fit_transform(x_train)
#x_tr_v = Count_Vec.fit_transform(x_tr)

In [480]:
# Count_Vec.get_feature_names()

In [481]:
x_test_v = Count_Vec.transform(x_test)
#x_te_v = Count_Vec.transform(x_te)

In [482]:
#svc = SVC()
rfc = RandomForestClassifier()
Multi = MultinomialNB(alpha=0.27)
#svc.fit(x_train_v, y_train)

USING CROSS VALIDATION FOR DETERMINING THE SCORE

In [485]:
from sklearn.model_selection import cross_val_score, KFold

In [486]:
# cv = KFold(n_splits= 4, shuffle= False, random_state= 1)
# cross_val_score(Multi, x_train_v, y_train, scoring = 'accuracy', cv = cv, verbose = 1)

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished


array([0.771949  , 0.76065574, 0.76211293, 0.77340619])

In [487]:
Multi.fit(x_train_v, y_train)

MultinomialNB(alpha=0.27, class_prior=None, fit_prior=True)

In [488]:
y_pred = Multi.predict(x_test_v)

In [489]:
np.savetxt(r'C:\Users\asus\Downloads\TwitterSentiment\ypreddcop.csv', y_pred, delimiter=',', fmt="%s")